In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1,2'
import numpy as np
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision

from data.dataset import PascalVOCDataset

%matplotlib inline

ModuleNotFoundError: No module named 'torch'

In [2]:
train_data_opt = {'info_path': '/home/stevetod/jzy/dataset/PascalVOC/train_anno.txt',
             'root_path': '/home/stevetod/jzy/dataset/PascalVOC/JPEGImages/',
             'phase': 'train',
             'input_size': 224,
             'category_num': 20}
val_data_opt = {'info_path': '/home/stevetod/jzy/dataset/PascalVOC/val_anno.txt',
             'root_path': '/home/stevetod/jzy/dataset/PascalVOC/JPEGImages/',
             'phase': 'val',
             'input_size': 224,
             'category_num': 20}

In [3]:
train_ds = PascalVOCDataset(train_data_opt)
val_ds = PascalVOCDataset(val_data_opt)

In [4]:
train_dl = DataLoader(train_ds, batch_size=32, num_workers=8, pin_memory=True)
val_dl = DataLoader(val_ds)

In [ ]:
            
hist_dict = {'mAcc': [], 'wAcc':[], 'f1': []}
result_array = np.zeros((1, 20), dtype=int)
cnt_array = np.zeros((1, 20), dtype=int)
TP_array = np.zeros((1, 20), dtype=int)
FP_FN_array = np.zeros((1, 20), dtype=int)

for feature in range(20):
    model_ft = torchvision.models.densenet161(pretrained=True)
    num_ftrs = model_ft.classifier.in_features
    model_ft.classifier = nn.Sequential(nn.Linear(num_ftrs, 1), nn.Sigmoid())
    
    #input_size = 224
    optimizer = torch.optim.SGD(model_ft.parameters(), lr=1e-4, weight_decay=1e-4)
    criterion = nn.BCELoss().cuda()
    model_ft = model_ft.cuda()
    
    for epoch in range(10):
        model_ft.train()
        with tqdm(total = len(train_dl)) as t:
            loss_hist = []
            for i, sample in enumerate(train_dl):
                optimizer.zero_grad()
                img, target = sample
                target = target[:,feature] # single feature
                img = img.cuda()
                target = target.cuda()
                output = model_ft(img)
                #print(output.size())
                loss = criterion(output, target)
                loss.backward()
                optimizer.step()
                t.set_postfix_str("Batch Loss: %.4f" % loss.item())
                t.update()
                loss_hist.append(loss.item())
        print('Mean train loss: %.4f' % np.mean(loss_hist))    
        
    model_ft.eval()
    
    with torch.no_grad():
        for i, sample in tqdm(enumerate(val_dl), total=len(val_dl)):
            img, target = sample
            target = target[:,feature]
            img = img.cuda()
            target = target.numpy().astype(int)
            output = model_ft(img)
            output = output.cpu().numpy() > 0.5
            result = (target.astype(bool) == output).astype(int) # if true result=1 else res=0
            TP_array[0,feature] += np.logical_and(target.astype(bool), output).astype(int).item()
            FP_FN_array[0,feature] += np.logical_xor(target.astype(bool), output).astype(int).item()
            result_array[0,feature] += result.item()
            cnt_array[0,feature] += target.item()
                
    mAcc = np.mean(result_array[0] / len(val_dl))
    wAcc = np.sum(result_array / len(val_dl) * cnt_array / np.sum(cnt_array))
    f1 = 2.0 * TP_array / (2.0 * TP_array + FP_FN_array)
    f1[np.isnan(f1)] = 0
    hist_dict['mAcc'].append(mAcc)
    hist_dict['wAcc'].append(wAcc)
    hist_dict['f1'].append(f1)
    print('mAcc: ', np.mean(result_array[0] / len(val_dl)))
    print('wAcc: ', np.sum(result_array / len(val_dl) * cnt_array / np.sum(cnt_array)))
    print('F1-score', ', '.join(['%.2f' % x for x in f1[0]]))        
        

/home/data/private/disk5/stevetod/anaconda3/envs/pytorch_py3/lib/python3.6/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
/home/data/private/disk5/stevetod/anaconda3/envs/pytorch_py3/lib/python3.6/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))



Mean train loss: 0.5945



Mean train loss: 0.6349



Mean train loss: 0.6290



Mean train loss: 0.6179


...


Mean train loss: 0.1564



Mean train loss: 0.1480



Mean train loss: 0.1427



Mean train loss: 0.1375



Mean train loss: 0.1321


In [5]:
model_ft = torchvision.models.densenet161(pretrained=True)
num_ftrs = model_ft.classifier.in_features
model_ft.classifier = nn.Sequential(nn.Linear(num_ftrs, 20), nn.Sigmoid())
input_size = 224

In [6]:
optimizer = torch.optim.SGD(model_ft.parameters(), lr=1e-4, weight_decay=1e-4)
criterion = nn.BCELoss().cuda()
model_ft = model_ft.cuda()

In [ ]:
hist_dict = {'mAcc': [], 'wAcc':[], 'f1': []}
for e in range(10):
    model_ft.train()
    with tqdm(total=len(train_dl)) as t:
        loss_hist = []
        for i, sample in enumerate(train_dl):
            optimizer.zero_grad()
            img, target = sample
            img = img.cuda()
            target = target.cuda()
            output = model_ft(img)
            print(target.size(),output.size())
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            t.set_postfix_str("Batch Loss: %.4f" % loss.item())
            t.update()
            loss_hist.append(loss.item())
    print('Mean train loss: %.4f' % np.mean(loss_hist))
    
    # val
    model_ft.eval()
    result_array = np.zeros((1, 20), dtype=int)
    cnt_array = np.zeros((1, 20), dtype=int)
    TP_array = np.zeros((1, 20), dtype=int)
    FP_FN_array = np.zeros((1, 20), dtype=int)
    with torch.no_grad():
        for i, sample in tqdm(enumerate(val_dl), total=len(val_dl)):
            img, target = sample
            img = img.cuda()
            target = target.numpy().astype(int)
            output = model_ft(img)
            output = output.cpu().numpy() > 0.5
            res = (target.astype(bool) == output).astype(int)
            TP_array += np.logical_and(target.astype(bool), output).astype(int)
            FP_FN_array += np.logical_xor(target.astype(bool), output).astype(int)
            result_array += res
            cnt_array += target
        mAcc = np.mean(result_array[0] / len(val_dl))
        wAcc = np.sum(result_array / len(val_dl) * cnt_array / np.sum(cnt_array))
        f1 = 2.0 * TP_array / (2.0 * TP_array + FP_FN_array)
        hist_dict['mAcc'].append(mAcc)
        hist_dict['wAcc'].append(wAcc)
        hist_dict['f1'].append(f1)
        print('mAcc: ', np.mean(result_array[0] / len(val_dl)))
        print('wAcc: ', np.sum(result_array / len(val_dl) * cnt_array / np.sum(cnt_array)))
        print('F1-score', ', '.join(['%.2f' % x for x in f1[0]]))

torch.Size([32, 20]) torch.Size([32, 20])
torch.Size([32, 20]) torch.Size([32, 20])
torch.Size([32, 20]) torch.Size([32, 20])
torch.Size([32, 20]) torch.Size([32, 20])
torch.Size([32, 20]) torch.Size([32, 20])
torch.Size([32, 20]) torch.Size([32, 20])
torch.Size([32, 20]) torch.Size([32, 20])
torch.Size([32, 20]) torch.Size([32, 20])
torch.Size([32, 20]) torch.Size([32, 20])
torch.Size([32, 20]) torch.Size([32, 20])
torch.Size([32, 20]) torch.Size([32, 20])
torch.Size([32, 20]) torch.Size([32, 20])
torch.Size([32, 20]) torch.Size([32, 20])
torch.Size([32, 20]) torch.Size([32, 20])
torch.Size([32, 20]) torch.Size([32, 20])
torch.Size([32, 20]) torch.Size([32, 20])


In [50]:
#plt.bar(np.arange(1, 21), result_array[0] / len(val_dl))
mAcc = np.mean(result_array[0] / len(val_dl))
wAcc = np.sum(result_array / len(val_dl) * cnt_array / np.sum(cnt_array))
f1 = 2.0 * TP_array / (2.0 * TP_array + FP_FN_array)
print('mAcc: ', np.mean(result_array[0] / len(val_dl)))
print('wAcc: ', np.sum(result_array / len(val_dl) * cnt_array / np.sum(cnt_array)))
print('F1-score', ', '.join(['%.2f' % x for x in f1[0]]))
#plt.bar(np.arange(1, 21), f1[0])
#t = plt.title('F1-score')

mAcc:  0.9427680798004987
wAcc:  0.7780759072782178
F1-score 0.73, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00
